<a href="https://colab.research.google.com/github/KeneKing12/Kenechukwu/blob/main/PLASTIC_POLLUTANT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, applications
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import cv2

# Define dataset path
data_dir = "C:/Users/hp/Documents/CANVAS STUDY- UNI SW/HT25-LP1-DIL700-Deep Learning/TERM PROJECT/CODES/MARIDA/patches"

# Load images and labels
def load_data(data_dir, img_size=(224, 224)):
    images, labels = [], []
    for category in os.listdir(data_dir): # Assume subfolders represent classes
        category_path = os.path.join(data_dir, category)
        if os.path.isdir(category_path):
            for img_name in os.listdir(category_path):
                img_path = os.path.join(category_path, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, img_size)
                    images.append(img)
                    labels.append(category)
    return np.array(images), np.array(labels)

# Load data
images, labels = load_data(data_dir)
labels_encoded = LabelEncoder().fit_transform(labels)
images = images / 255.0  # Normalize

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)

# CNN Model Definition
def build_cnn_model(input_shape=(224, 224, 3)):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')  # Binary Classification
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

cnn_model = build_cnn_model()

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2)

datagen.fit(X_train)

# Train CNN Model
cnn_model.fit(datagen.flow(X_train, y_train, batch_size=32),
              validation_data=(X_test, y_test),
              epochs=10)

# Transfer Learning using EfficientNet
base_model = applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

def build_transfer_learning_model():
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

transfer_model = build_transfer_learning_model()
transfer_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

# Train Random Forest for comparison
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_flat, y_train)

# Evaluate Models
cnn_preds = (cnn_model.predict(X_test) > 0.5).astype(int)
tf_preds = (transfer_model.predict(X_test) > 0.5).astype(int)
rf_preds = rf_model.predict(X_test_flat)

print("CNN Model Evaluation:")
print(confusion_matrix(y_test, cnn_preds))
print(classification_report(y_test, cnn_preds))

print("Transfer Learning Model Evaluation:")
print(confusion_matrix(y_test, tf_preds))
print(classification_report(y_test, tf_preds))

print("Random Forest Model Evaluation:")
print(confusion_matrix(y_test, rf_preds))
print(classification_report(y_test, rf_preds))


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/hp/Documents/CANVAS STUDY- UNI SW/HT25-LP1-DIL700-Deep Learning/TERM PROJECT/CODES/MARIDA/patches'